In [83]:
#Dependencies
import pandas as pd
import requests
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time

### NASA Mars News 

In [52]:
#set url
url_news = 'https://mars.nasa.gov/news/'

In [54]:
#set exe path and browser
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(url_news)

In [55]:
# HTML object
html_news = browser.html
# Create BeautifulSoup object; parse with lxml
soupy = bs(html_news, 'lxml')

In [56]:
# Retrieve the latest news title and paragraph
title = soupy.find('div', class_="bottom_gradient").h3.text
para = soupy.find('div', class_="article_teaser_body").text
date = soupy.find('div', class_="list_date").text
print('-------------------')
print(f'{title}:') 
print('-------------------')
print(para) 
print('-------------------')
print(f'Published: {date}')

-------------------
Hear Audio From NASA's Perseverance As It Travels Through Deep Space:
-------------------
The first to be rigged with microphones, the agency's latest Mars rover picked up the subtle sounds of its own inner workings during interplanetary flight.
-------------------
Published: November 18, 2020


### JPL Mars Space Images - Featured Image

In [162]:
#set search url
url_img = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

In [179]:
# set base feature image url
base_img_url ='https://www.jpl.nasa.gov/spaceimages/images/largesize/'

In [164]:
#set exe path and browser
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(url_img)

In [165]:
# HTML object
html_img = browser.html
# Create BeautifulSoup object; parse with lxml
soupimg = bs(html_img, 'lxml')

In [166]:
# retrieve current Featured Mars Image
image_find = soupimg.find('a', class_='button fancybox')['data-fancybox-href']
image_pg = (f'{url_img}{image_find}')
print(image_pg)


https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars/spaceimages/images/mediumsize/PIA18906_ip.jpg


In [168]:
#visit next page for feature image name and blurb
browser.visit(image_pg)

In [172]:
#get name and blurb about feature image
image_name = soupimg.find('a', class_='button fancybox')['data-title']
image_desc = soupimg.find('a', class_='button fancybox')['data-description']

print(image_name)
print(image_desc)

Sun Shines in High-Energy X-rays
X-rays stream off the sun in this first picture of the sun, overlaid on a picture taken by NASA's Solar Dynamics Observatory, taken by NuSTAR.


In [180]:
# get feature id by it's string position and make feature img url
jpg = 'hires.jpg'
end=len(image_find)-6
start1= image_find.find("z")
start=start1+3
img_id=image_find[start:end]
feature_image_url = f'{base_img_url}{img_id}{jpg}'
print(feature_image_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18906_hires.jpg


### Mars Facts

In [ ]:
# set url
url_facts = 'https://space-facts.com/mars/'

In [ ]:
#Use Panda's `read_html` to parse the url
table = pd.read_html(url_facts)
#table

In [ ]:
# create df of facts
mars_facts_df = table[2]
mars_facts_df.columns = ["What", "Fact"]
mars_facts_df.set_index("What", inplace=True)
mars_facts_df

In [ ]:
# convert df to html table string
mars_html_table1 = mars_facts_df.to_html()
#mars_html_table1

In [ ]:
# remove carrige return \n
mars_html_table = mars_html_table1.replace('\n', '')
print(mars_html_table)

### Mars Hemispheres

In [182]:
#Set url
url_hem = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

In [193]:
#set base url to append hemi image download tif file name to
base_hemi_url='https://astropedia.astrogeology.usgs.gov/download'

In [183]:
#set exe path and browser
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(url_hem)

In [184]:
# HTML object
html_hem = browser.html
# Create BeautifulSoup object; parse with lxml
souphem = bs(html_hem, 'lxml')

In [197]:
# loop to gather all full res hemisphere img urls
start_url = 'https://astrogeology.usgs.gov'
hemisphere_image_urls = []

# Retrieve the parent divs for all articles
results_all = souphem.find('div', class_='collapsible results')
hemispheres = results_all.find_all('div', class_='item')

# loop over results to get hemi title and img url
for hemi in hemispheres:
    # scrape the link hemi title
    title_hemi = hemi.find('div', class_='description').h3.text
    # scrape the link to hemi img
    img_url = hemi.find('div', class_='description').a["href"]
    #get end of url for tif download file
    start_hemi = img_url.find("p/")+1
    end_hemi = len(img_url)
    for_tif=img_url[start_hemi:end_hemi]
    img_hemi_url=f'{base_hemi_url}{for_tif}.tif'
    
    
    # make title and img_url into dict and store in list
    hemi_dict = {}
    hemi_dict['title'] = title_hemi
    hemi_dict['img_url'] = img_hemi_url
    
    hemisphere_image_urls.append(hemi_dict)

print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif'}]


###  Create a dictionary of all the scraped mars data

In [ ]:
# Creating a dictionary that holds all the scraped data
mars_scrpd = {}
mars_scrpd["news_title"] = title
mars_scrpd["news_paragraph"] = para
mars_scrpd["news_date"] = date
mars_scrpd["feature_image_url"] = feature_image_url
mars_scrpd["feature_image_name"] = image_name
mars_scrpd["feature_image_described"] = image_desc
mars_scrpd["mars_facts"] = mars_html_table
mars_scrpd["hemisphere_image_urls"] = hemisphere_image_urls
mars_scrpd